In [ ]:
import numpy as np 
import math as m 
import pandas as pd 
import itertools as iter 

In [ ]:
kordinate = pd.read_csv('Kordinate.csv')
kordinate = kordinate.to_numpy()
print(kordinate)

nivelanje = pd.read_csv('Nivelanje.csv')
nivelanje=nivelanje.to_numpy()
nivelanje=nivelanje-1
print(nivelanje)

In [ ]:
try:
    sigma_h = float(input("Kolika je zahtevana tacnost nivelanja "))
except:
    print("Niste dobro uneli broj")
    sigma_h = float(input("Kolika je zahtevana tacnost nivelanja "))
    
try:
    datum_reper = int(input("Kojim reperom je definisan datum "))
except:
    print("Niste dobro uneli broj")
    datum_reper = int(input("Kojim reperom je definisan datum "))
    
try:
    sigma_usvojeno = float(input("Kolika je usvojena tacnost nivelanja "))
except:
    print("Niste dobro uneli broj")
    sigma_usvojeno = float(input("Kolika je usvojena tacnost nivelanja "))

In [ ]:
def kolonaA(arr):
    kolonaA = np.zeros((1,len(np.unique(nivelanje))),dtype = float)
    kolonaA[0,arr[0]],kolonaA[0,arr[1]]= -1,1
    return kolonaA
def matricaA(nivelanje):
    arr = (kolonaA(i) for i in nivelanje)
    A = np.vstack(tuple(arr))
    return A


In [ ]:
A =matricaA(nivelanje)
A

In [ ]:
def matricaP(nivelanje):
    var =[]
    for i in duzine:
        
    var = np.array(var)
    devijacije =var*sigma_h
    P = 1/((var*sigma_h)**2)
    P = np.diag(P)  
    return  P ,devijacije
 


In [ ]:
P,devijacije = matricaP(nivelanje)
np.diag(P)

In [ ]:
def matricaRt():
    Rt = np.zeros((len(np.unique(nivelanje))))
    Rt[datum_reper-1]=1
    Rt.reshape(1,7)
    return Rt
    
Rt = matricaRt()
Rt.shape

In [ ]:
def matricaN(A,P,Rt):
    N = np.matmul(np.matmul(np.transpose(A),P),A)
    N=np.hstack((np.vstack((N,Rt)),np.append(Rt.transpose(),0).reshape(-1,1)))
    return N



In [ ]:
N = matricaN(A,P,Rt)

In [ ]:
def analizaTacnosti(A,P,N):
    Qx = np.linalg.inv(N)
    Qx = Qx[:-1,:-1]
    Qhi = np.diag(Qx)
    sigma_hi = sigma_usvojeno*np.sqrt(Qhi)
    
    Ql = np.matmul(np.matmul(A,Qx),A.transpose())
    Qv = np.linalg.inv(P)-Ql
    ri = np.diag(P)*np.diag(Qv)
    Gi = 2.802/(np.diag(P)*np.sqrt(np.diag(Qv)))
    
    return Qx,Qhi,sigma_hi,Ql,Qv,ri,Gi,np.sum(ri)
    

In [ ]:
def analizaTacnostiN(A,P,N):
    Qx = np.linalg.inv(N)
    Qhi = np.diag(Qx)
    sigma_hi = sigma_usvojeno*np.sqrt(Qhi)
    
    Ql = np.matmul(np.matmul(A,Qx),A.transpose())
    Qv = np.linalg.inv(P)-Ql
    ri = np.diag(P)*np.diag(Qv)
    Gi = 2.802/(np.diag(P)*np.sqrt(np.diag(Qv)))
    
    return Qx,Qhi,sigma_hi,Ql,Qv,ri,Gi,np.sum(ri)

In [ ]:
Qx,Qhi,sigma_hi,Ql,Qv,ri,Gi,kontrola = analizaTacnosti(A,P,N)



## Izravnanje sa minimalnim tragom

In [ ]:
Bt = np.full((1,len(np.unique(nivelanje))),1/m.sqrt(kordinate.shape[0]))
Bt.shape

In [ ]:
bN = matricaN(A,P,Bt)

In [ ]:
bQx,bQhi,bsigma_hi,bQl,bQv,bri,bGi,bkontrola = analizaTacnosti(A,P,bN)

## Neslobodna mreza 

In [ ]:
fiksirane_tacke = []
f_tacka = 1
while(True):
    
    f_tacka = int(input("Tacka koja je fiksirana"))
    if f_tacka ==0:
        break
    fiksirane_tacke.append(f_tacka-1)


In [ ]:
def fiksiraniAP(fiksirane_tacke,A,P):
    perm = iter.permutations(fiksirane_tacke,r=2)
    perm = list(perm)
    index = []
    for idx,i in enumerate(nivelanje):
        if tuple(i) in perm:
            index.append(idx)
    
    mask = np.ones(len(P), dtype=bool)
    mask[index] = False
    ndevijacije = devijacije[mask]
    nP = np.diag(P)[mask]
    nP = np.diag(nP)
    
    
    nA = A.copy()
    nA = np.delete(nA,fiksirane_tacke,axis = 1)
    nA = np.delete(nA,index,axis = 0)
    
    nN = np.matmul(np.matmul(np.transpose(nA),nP),nA)
    
    return nA,ndevijacije,nP,nN

In [ ]:
nA,ndevijacije,nP,nN = fiksiraniAP(fiksirane_tacke,A,P)
print(nA.shape,nP.shape,nN.shape)


In [ ]:
ndevijacije.round(2)

In [ ]:
nQx,nQhi,nsigma_hi,nQl,nQv,nri,nGi,nkontrola = analizaTacnostiN(nA,nP,nN)

In [ ]:
nQx = np.linalg.inv(nN)
nQhi = np.diag(nQx)
nsigma_hi = sigma_usvojeno*np.sqrt(nQhi)

nQl = np.matmul(np.matmul(nA,nQx),nA.transpose())
nQv = np.linalg.inv(nP)-nQl
nri = np.diag(nP)*np.diag(nQv)
nGi = 2.802/(np.diag(nP)*np.sqrt(np.diag(nQv)))
pd.DataFrame(nQv)

In [ ]:
Af = pd.DataFrame(A.round(3))
Pf = pd.DataFrame(P.round(3))
Nf = pd.DataFrame(N.round(3))
Rtf = pd.DataFrame(Rt.round(3))
Qxf = pd.DataFrame(Qx.round(3))
sigma_hif = pd.DataFrame(sigma_hi.round(3))
Qvf = pd.DataFrame(Qv.round(3))
rif = pd.DataFrame(ri.round(3))
Gif = pd.DataFrame(Gi.round(3))


bNf = pd.DataFrame(bN.round(3))
Btf = pd.DataFrame(Bt.round(3))
bQxf = pd.DataFrame(bQx.round(3))
bsigma_hif = pd.DataFrame(bsigma_hi.round(3))
bQvf = pd.DataFrame(bQv.round(3))
brif = pd.DataFrame(bri.round(3))
bGif = pd.DataFrame(bGi.round(3))

nAf = pd.DataFrame(nA.round(3))
nPf = pd.DataFrame(nP.round(3))
nNf = pd.DataFrame(nN.round(3))
nQxf = pd.DataFrame(nQx.round(3))
nsigma_hif = pd.DataFrame(nsigma_hi.round(3))
nQvf = pd.DataFrame(nQv.round(3))
nrif = pd.DataFrame(nri.round(3))
nGif = pd.DataFrame(nGi.round(3))

In [ ]:
dfs = {"Matrica A":Af,'Matrica P ' :Pf,"Matrica N":Nf,"Matrica Rt":Rtf,"Matrica Qx":Qxf,"Mhi":sigma_hif,"Matrica Qv ":Qvf,
       "Elementi ri":rif,"Elementi Gi":Gif,
      "Matrica bN":bNf,"Matrica Bt":Btf,"Matrica bQx":bQxf,"bMhi":bsigma_hif,"bMatrica Qv":bQvf,
       "Elementi bri":brif,"bElementi bGi":bGif,
      "Matrica nA":nAf,'Matrica nP ' :nPf,"Matrica nN":nNf,"Matrica nQx":nQxf,"nMhi":nsigma_hif,"nMatrica Qv ":nQvf,
       "Elementi nri":nrif,"Elementi nGi":nGif}

writer = pd.ExcelWriter('resenja2.xlsx', engine='xlsxwriter')
for sheet_name in dfs.keys():
    dfs[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False,header = False)
    
writer.save()

In [ ]:
nGi